In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
from scipy import stats
import warnings
warnings.filterwarnings("ignore")

pd.set_option("display.max_column",100)
pd.set_option("expand_frame_repr",False)
%matplotlib inline

In [2]:
#1--读取数据
test_data=pd.read_csv(r'D:\文件\学习\数据集\天猫用户复购\data_format1\test_format1.csv')
train_data=pd.read_csv(r'D:\文件\学习\数据集\天猫用户复购\data_format1\train_format1.csv')
user_info=pd.read_csv(r'D:\文件\学习\数据集\天猫用户复购\data_format1\user_info_format1.csv')
user_log=pd.read_csv(r'D:\文件\学习\数据集\天猫用户复购\data_format1\user_log_format1.csv')

In [3]:
import gc
from collections import Counter
import copy

In [4]:
#内存压缩
def reduce_mem_usage(df,verbose=True):
    start_mem=df.memory_usage().sum()/1024**2
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    for col in df.columns:
        col_type=df[col].dtype
        if col_type in numerics:
            c_min=df[col].min()
            c_max=df[col].max()
            if str(col_type)[:3] =='int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col]=df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col]=df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col]=df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col]=df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col]=df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col]=df[col].astype(np.float64)
                elif c_min > np.finfo(np.float64).min and c_max < np.finfo(np.float64).max:
                    df[col]=df[col].astype(np.float64)
    end_mem=df.memory_usage().sum()/1024**2
    print("压缩前：",start_mem)
    print("压缩后：",end_mem)
    return df

In [5]:
train_data=reduce_mem_usage(train_data)
test_data=reduce_mem_usage(test_data)
user_info=reduce_mem_usage(user_info)
user_log=reduce_mem_usage(user_log)

压缩前： 5.9708251953125
压缩后： 1.7415771484375
压缩前： 5.984855651855469
压缩后： 3.4912166595458984
压缩前： 9.708602905273438
压缩后： 3.2362823486328125
压缩前： 2933.3292083740234
压缩后： 890.4750232696533


In [6]:
#合并数据
all_data=train_data.append(test_data)
all_data=all_data.merge(user_info,on=["user_id"],how="left")

In [7]:
#排序
user_log=user_log.sort_values(["user_id","time_stamp"])
user_log.head(3)

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type
23288890,1,181459,276,2245,4752.0,1009,0
23288891,1,779078,276,2245,4752.0,1009,0
23288892,1,779078,276,2245,4752.0,1009,0


In [8]:
#合并数据
list_join_func=lambda x:" ".join([str(i) for i in x])

In [9]:
agg_dict={
    "item_id":list_join_func,
    "cat_id":list_join_func,
    "seller_id":list_join_func,
    "brand_id":list_join_func,
    "time_stamp":list_join_func,
    "action_type":list_join_func
}

In [10]:
rename_dict={
    "item_id":"item_path",
    "cat_id":"cat_path",
    "seller_id":"seller_path",
    "brand_id":"brand_path",
    "time_stamp":"time_stamp_path",
    "action_type":"action_type_path"
}

In [11]:
user_log_path=user_log.groupby("user_id").agg(agg_dict).reset_index().rename(columns=rename_dict)

In [12]:
user_log_path.head()

,user_id,item_path,cat_path,seller_path,brand_path,time_stamp_path,action_type_path
0,1,181459 779078 779078 452837 543397 504149 5041...,276 276 276 276 276 1023 1023 1023 1023 1252 1...,2245 2245 2245 2245 2245 925 925 925 925 4026 ...,4752.0 4752.0 4752.0 4752.0 4752.0 7400.0 7400...,1009 1009 1009 1009 1009 1011 1011 1011 1011 1...,0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 0 0 0 0 0 0 2 2 ...
1,2,348983 749563 239288 751744 239288 714176 1972...,177 177 602 602 602 1213 602 602 602 1213 1213...,2223 2223 420 420 420 420 420 420 420 420 420 ...,3272.0 3272.0 4952.0 4952.0 4952.0 4058.0 4952...,527 527 626 626 626 626 626 626 626 626 626 62...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 0 0 ...
2,3,895754 895754 182882 182882 985337 175397 9463...,1505 1505 1271 1271 1271 662 1134 1134 1134 45...,795 795 2123 2123 4925 1102 4461 4461 4461 474...,3608.0 3608.0 4796.0 4796.0 8004.0 1214.0 905....,516 516 627 627 627 727 819 819 820 906 906 90...,2 2 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
3,4,836727 243874 185489 95474 30073 790055 790055...,612 1577 1505 1505 1505 1505 1505 1505 1505 15...,1221 1221 1221 1221 1221 1221 1221 1221 1221 1...,7736.0 7736.0 7736.0 7736.0 7736.0 7736.0 7736...,527 527 527 527 527 527 527 527 527 527 527 52...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
4,5,511956 728354 568450 754044 1011255 741215 362...,302 351 812 1213 142 464 1028 35 1213 351 142 ...,3098 3215 641 3736 1483 176 4848 641 3736 4547...,5544.0 5480.0 4264.0 3124.0 4640.0 6664.0 2804...,519 520 520 520 520 520 520 520 520 520 520 52...,3 0 0 2 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 ...


In [13]:
all_data_path=all_data.merge(user_log_path,on="user_id",how="left")
all_data_path.head()

,user_id,merchant_id,label,prob,age_range,gender,item_path,cat_path,seller_path,brand_path,time_stamp_path,action_type_path
0,34176,3906,0.0,NaN,6.0,0.0,581818 879005 581818 581818 1011673 52343 2773...,1505 662 1505 1505 1505 662 1095 1505 662 1095...,416 3606 416 416 416 3760 3606 416 1926 3004 4...,4014.0 33.0 4014.0 4014.0 4014.0 3738.0 33.0 4...,521 521 521 521 521 521 521 521 521 521 521 52...,0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 0 2 ...
1,34176,121,0.0,NaN,6.0,0.0,581818 879005 581818 581818 1011673 52343 2773...,1505 662 1505 1505 1505 662 1095 1505 662 1095...,416 3606 416 416 416 3760 3606 416 1926 3004 4...,4014.0 33.0 4014.0 4014.0 4014.0 3738.0 33.0 4...,521 521 521 521 521 521 521 521 521 521 521 52...,0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 0 2 ...
2,34176,4356,1.0,NaN,6.0,0.0,581818 879005 581818 581818 1011673 52343 2773...,1505 662 1505 1505 1505 662 1095 1505 662 1095...,416 3606 416 416 416 3760 3606 416 1926 3004 4...,4014.0 33.0 4014.0 4014.0 4014.0 3738.0 33.0 4...,521 521 521 521 521 521 521 521 521 521 521 52...,0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 0 2 ...
3,34176,2217,0.0,NaN,6.0,0.0,581818 879005 581818 581818 1011673 52343 2773...,1505 662 1505 1505 1505 662 1095 1505 662 1095...,416 3606 416 416 416 3760 3606 416 1926 3004 4...,4014.0 33.0 4014.0 4014.0 4014.0 3738.0 33.0 4...,521 521 521 521 521 521 521 521 521 521 521 52...,0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 0 2 ...
4,230784,4818,0.0,NaN,0.0,0.0,191923 191923 191923 191923 964906 229470 2294...,1023 1023 1023 1023 662 664 664 1544 664 662 6...,3545 3545 3545 3545 4566 2537 2537 2420 2537 4...,5860.0 5860.0 5860.0 5860.0 6320.0 6064.0 6064...,601 601 601 601 614 614 614 614 614 614 618 61...,0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 0 0 0 0 0 ...


In [14]:
all_data_path=all_data_path.sort_values(["user_id"])

In [15]:
all_data_path.head()

,user_id,merchant_id,label,prob,age_range,gender,item_path,cat_path,seller_path,brand_path,time_stamp_path,action_type_path
130649,1,1019,1.0,NaN,3.0,1.0,181459 779078 779078 452837 543397 504149 5041...,276 276 276 276 276 1023 1023 1023 1023 1252 1...,2245 2245 2245 2245 2245 925 925 925 925 4026 ...,4752.0 4752.0 4752.0 4752.0 4752.0 7400.0 7400...,1009 1009 1009 1009 1009 1011 1011 1011 1011 1...,0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 0 0 0 0 0 0 2 2 ...
479611,2,1784,NaN,NaN,3.0,0.0,348983 749563 239288 751744 239288 714176 1972...,177 177 602 602 602 1213 602 602 602 1213 1213...,2223 2223 420 420 420 420 420 420 420 420 420 ...,3272.0 3272.0 4952.0 4952.0 4952.0 4058.0 4952...,527 527 626 626 626 626 626 626 626 626 626 62...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 0 0 ...
479612,2,1679,NaN,NaN,3.0,0.0,348983 749563 239288 751744 239288 714176 1972...,177 177 602 602 602 1213 602 602 602 1213 1213...,2223 2223 420 420 420 420 420 420 420 420 420 ...,3272.0 3272.0 4952.0 4952.0 4952.0 4058.0 4952...,527 527 626 626 626 626 626 626 626 626 626 62...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 0 0 ...
305759,3,2313,NaN,NaN,3.0,0.0,895754 895754 182882 182882 985337 175397 9463...,1505 1505 1271 1271 1271 662 1134 1134 1134 45...,795 795 2123 2123 4925 1102 4461 4461 4461 474...,3608.0 3608.0 4796.0 4796.0 8004.0 1214.0 905....,516 516 627 627 627 727 819 819 820 906 906 90...,2 2 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
131623,4,1186,0.0,NaN,0.0,0.0,836727 243874 185489 95474 30073 790055 790055...,612 1577 1505 1505 1505 1505 1505 1505 1505 15...,1221 1221 1221 1221 1221 1221 1221 1221 1221 1...,7736.0 7736.0 7736.0 7736.0 7736.0 7736.0 7736...,527 527 527 527 527 527 527 527 527 527 527 52...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...


In [16]:
#删除重复项
all_data_path=all_data_path.drop_duplicates("user_id")

In [17]:
all_data_path.head(3)

,user_id,merchant_id,label,prob,age_range,gender,item_path,cat_path,seller_path,brand_path,time_stamp_path,action_type_path
130649,1,1019,1.0,NaN,3.0,1.0,181459 779078 779078 452837 543397 504149 5041...,276 276 276 276 276 1023 1023 1023 1023 1252 1...,2245 2245 2245 2245 2245 925 925 925 925 4026 ...,4752.0 4752.0 4752.0 4752.0 4752.0 7400.0 7400...,1009 1009 1009 1009 1009 1011 1011 1011 1011 1...,0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 0 0 0 0 0 0 2 2 ...
479611,2,1784,NaN,NaN,3.0,0.0,348983 749563 239288 751744 239288 714176 1972...,177 177 602 602 602 1213 602 602 602 1213 1213...,2223 2223 420 420 420 420 420 420 420 420 420 ...,3272.0 3272.0 4952.0 4952.0 4952.0 4058.0 4952...,527 527 626 626 626 626 626 626 626 626 626 62...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 0 0 ...
305759,3,2313,NaN,NaN,3.0,0.0,895754 895754 182882 182882 985337 175397 9463...,1505 1505 1271 1271 1271 662 1134 1134 1134 45...,795 795 2123 2123 4925 1102 4461 4461 4461 474...,3608.0 3608.0 4796.0 4796.0 8004.0 1214.0 905....,516 516 627 627 627 727 819 819 820 906 906 90...,2 2 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...


In [18]:
#定义统计函数
def cnt_(x):
    try:
        return len(x.split(" "))
    except:
        return -1

In [19]:
def nunique_(x):
    try:
        return len(set(x.split(" ")))
    except:
        return -1

In [20]:
def max_(x):
    try:
        return np.max([int(i) for i in x.split(" ")])
    except:
        return -1

In [21]:
def min_(x):
    try:
        return np.min([int(i) for i in x.split(" ")])
    except:
        return -1

In [22]:
def std_(x):
    try:
        return np.std([float(i) for i in x.split(" ")])
    except:
        return -1

In [23]:
def most_n(x,n):
    try:
        return Counter(x.split(" ")).most_common(n)[n-1][0]
    except:
        return -1

In [24]:
def most_n_cnt(x,n):
    try:
        return Counter(x.split(" ")).most_common(n)[n-1][1]
    except:
        return -1

In [25]:
def user_cnt(df_data,single_col,name):
    df_data[name]=df_data[single_col].apply(cnt_)
    return df_data

In [26]:
def user_nunique(df_data,single_col,name):
    df_data[name]=df_data[single_col].apply(nunique_)
    return df_data

In [27]:
def user_max(df_data,single_col,name):
    df_data[name]=df_data[single_col].apply(max_)
    return df_data

In [28]:
def user_min(df_data,single_col,name):
    df_data[name]=df_data[single_col].apply(min_)
    return df_data

In [29]:
def user_std(df_data,single_col,name):
    df_data[name]=df_data[single_col].apply(std_)
    return

In [30]:
def user_most_n(df_data,single_col,name,n=1):
    func=lambda x:most_n(x,n)
    df_data[name]=df_data[single_col].apply(func)
    return df_data

In [31]:
def user_most_n_cnt(df_data,single_col,name,n=1):
    func=lambda x:most_n_cnt(x,n)
    df_data[name]=df_data[single_col].apply(func)
    return df_data

In [32]:
all_data_test=all_data_path.head(2000)

In [33]:
all_data_test=user_cnt(all_data_test,"seller_path","user_cnt")
all_data_test.head(3)

,user_id,merchant_id,label,prob,age_range,gender,item_path,cat_path,seller_path,brand_path,time_stamp_path,action_type_path,user_cnt
130649,1,1019,1.0,NaN,3.0,1.0,181459 779078 779078 452837 543397 504149 5041...,276 276 276 276 276 1023 1023 1023 1023 1252 1...,2245 2245 2245 2245 2245 925 925 925 925 4026 ...,4752.0 4752.0 4752.0 4752.0 4752.0 7400.0 7400...,1009 1009 1009 1009 1009 1011 1011 1011 1011 1...,0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 0 0 0 0 0 0 2 2 ...,33
479611,2,1784,NaN,NaN,3.0,0.0,348983 749563 239288 751744 239288 714176 1972...,177 177 602 602 602 1213 602 602 602 1213 1213...,2223 2223 420 420 420 420 420 420 420 420 420 ...,3272.0 3272.0 4952.0 4952.0 4952.0 4058.0 4952...,527 527 626 626 626 626 626 626 626 626 626 62...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 0 0 ...,63
305759,3,2313,NaN,NaN,3.0,0.0,895754 895754 182882 182882 985337 175397 9463...,1505 1505 1271 1271 1271 662 1134 1134 1134 45...,795 795 2123 2123 4925 1102 4461 4461 4461 474...,3608.0 3608.0 4796.0 4796.0 8004.0 1214.0 905....,516 516 627 627 627 727 819 819 820 906 906 90...,2 2 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,68


In [34]:
all_data_test=user_nunique(all_data_path,"seller_path","seller_nunique")
all_data_test.head(3)

,user_id,merchant_id,label,prob,age_range,gender,item_path,cat_path,seller_path,brand_path,time_stamp_path,action_type_path,seller_nunique
130649,1,1019,1.0,NaN,3.0,1.0,181459 779078 779078 452837 543397 504149 5041...,276 276 276 276 276 1023 1023 1023 1023 1252 1...,2245 2245 2245 2245 2245 925 925 925 925 4026 ...,4752.0 4752.0 4752.0 4752.0 4752.0 7400.0 7400...,1009 1009 1009 1009 1009 1011 1011 1011 1011 1...,0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 0 0 0 0 0 0 2 2 ...,9
479611,2,1784,NaN,NaN,3.0,0.0,348983 749563 239288 751744 239288 714176 1972...,177 177 602 602 602 1213 602 602 602 1213 1213...,2223 2223 420 420 420 420 420 420 420 420 420 ...,3272.0 3272.0 4952.0 4952.0 4952.0 4058.0 4952...,527 527 626 626 626 626 626 626 626 626 626 62...,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 0 0 ...,14
305759,3,2313,NaN,NaN,3.0,0.0,895754 895754 182882 182882 985337 175397 9463...,1505 1505 1271 1271 1271 662 1134 1134 1134 45...,795 795 2123 2123 4925 1102 4461 4461 4461 474...,3608.0 3608.0 4796.0 4796.0 8004.0 1214.0 905....,516 516 627 627 627 727 819 819 820 906 906 90...,2 2 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...,23


In [35]:
all_data_test=user_nunique(all_data_path,"cat_path","cat_nunique")
all_data_test.head(1)

,user_id,merchant_id,label,prob,age_range,gender,item_path,cat_path,seller_path,brand_path,time_stamp_path,action_type_path,seller_nunique,cat_nunique
130649,1,1019,1.0,NaN,3.0,1.0,181459 779078 779078 452837 543397 504149 5041...,276 276 276 276 276 1023 1023 1023 1023 1252 1...,2245 2245 2245 2245 2245 925 925 925 925 4026 ...,4752.0 4752.0 4752.0 4752.0 4752.0 7400.0 7400...,1009 1009 1009 1009 1009 1011 1011 1011 1011 1...,0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 0 0 0 0 0 0 2 2 ...,9,6


In [36]:
all_data_test=user_nunique(all_data_path,"brand_path","brand_nunique")
all_data_test.head(1)

,user_id,merchant_id,label,prob,age_range,gender,item_path,cat_path,seller_path,brand_path,time_stamp_path,action_type_path,seller_nunique,cat_nunique,brand_nunique
130649,1,1019,1.0,NaN,3.0,1.0,181459 779078 779078 452837 543397 504149 5041...,276 276 276 276 276 1023 1023 1023 1023 1252 1...,2245 2245 2245 2245 2245 925 925 925 925 4026 ...,4752.0 4752.0 4752.0 4752.0 4752.0 7400.0 7400...,1009 1009 1009 1009 1009 1011 1011 1011 1011 1...,0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 0 0 0 0 0 0 2 2 ...,9,6,9


In [37]:
all_data_test=user_nunique(all_data_path,"item_path","item_numique")
all_data_test.head(1)

,user_id,merchant_id,label,prob,age_range,gender,item_path,cat_path,seller_path,brand_path,time_stamp_path,action_type_path,seller_nunique,cat_nunique,brand_nunique,item_numique
130649,1,1019,1.0,NaN,3.0,1.0,181459 779078 779078 452837 543397 504149 5041...,276 276 276 276 276 1023 1023 1023 1023 1252 1...,2245 2245 2245 2245 2245 925 925 925 925 4026 ...,4752.0 4752.0 4752.0 4752.0 4752.0 7400.0 7400...,1009 1009 1009 1009 1009 1011 1011 1011 1011 1...,0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 0 0 0 0 0 0 2 2 ...,9,6,9,12


In [38]:
all_data_test=user_nunique(all_data_test,"time_stamp_path","time_stamp_nunique")
all_data_test.head(1)

,user_id,merchant_id,label,prob,age_range,gender,item_path,cat_path,seller_path,brand_path,time_stamp_path,action_type_path,seller_nunique,cat_nunique,brand_nunique,item_numique,time_stamp_nunique
130649,1,1019,1.0,NaN,3.0,1.0,181459 779078 779078 452837 543397 504149 5041...,276 276 276 276 276 1023 1023 1023 1023 1252 1...,2245 2245 2245 2245 2245 925 925 925 925 4026 ...,4752.0 4752.0 4752.0 4752.0 4752.0 7400.0 7400...,1009 1009 1009 1009 1009 1011 1011 1011 1011 1...,0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 0 0 0 0 0 0 2 2 ...,9,6,9,12,5


In [39]:
all_data_test=user_nunique(all_data_test,"action_type_path","action_type_nunique")
all_data_test.head(1)

,user_id,merchant_id,label,prob,age_range,gender,item_path,cat_path,seller_path,brand_path,time_stamp_path,action_type_path,seller_nunique,cat_nunique,brand_nunique,item_numique,time_stamp_nunique,action_type_nunique
130649,1,1019,1.0,NaN,3.0,1.0,181459 779078 779078 452837 543397 504149 5041...,276 276 276 276 276 1023 1023 1023 1023 1252 1...,2245 2245 2245 2245 2245 925 925 925 925 4026 ...,4752.0 4752.0 4752.0 4752.0 4752.0 7400.0 7400...,1009 1009 1009 1009 1009 1011 1011 1011 1011 1...,0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 0 0 0 0 0 0 2 2 ...,9,6,9,12,5,2


In [40]:
#分开统计操作行为

In [41]:
all_data_test.head(1)

,user_id,merchant_id,label,prob,age_range,gender,item_path,cat_path,seller_path,brand_path,time_stamp_path,action_type_path,seller_nunique,cat_nunique,brand_nunique,item_numique,time_stamp_nunique,action_type_nunique
130649,1,1019,1.0,NaN,3.0,1.0,181459 779078 779078 452837 543397 504149 5041...,276 276 276 276 276 1023 1023 1023 1023 1252 1...,2245 2245 2245 2245 2245 925 925 925 925 4026 ...,4752.0 4752.0 4752.0 4752.0 4752.0 7400.0 7400...,1009 1009 1009 1009 1009 1011 1011 1011 1011 1...,0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 0 0 0 0 0 0 2 2 ...,9,6,9,12,5,2


In [43]:
action_type=all_data_test["action_type_path"].str.split(" ")
action_type

130649    [0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, ...
479611    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
305759    [2, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, ...
131623    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
480619    [3, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, ...
                                ...                        
470452    [2, 2, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 2, 2, 0, ...
35121     [0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 0, ...
384062    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
471378    [2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, ...
36151     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: action_type_path, Length: 424170, dtype: object

In [48]:
action_0=[]
action_1=[]
action_2=[]
action_3=[]
for col in action_type:
    action0=[]
    action1=[]
    action2=[]
    action3=[]
    for i in col:
        if i=="0":
            action0.append(i)
        elif i=="1":
            action1.append(i)
        elif i=="2":
            action2.append(i)
        elif i=="3":
            action3.append(i)
    action_0.append(len(action0))
    action_1.append(len(action1))
    action_2.append(len(action2))
    action_3.append(len(action3))